In [111]:
import pandas as pd
import tensorflow as tf
import numpy as np
import yfinance as yfin
import matplotlib.pyplot as plt
import seaborn as sns
import time

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Ibovespa

In [112]:
yfin.pdr_override()
ibovespa = yfin.download(['^BVSP'])
ibovespa

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1993-04-27,24.799999,25.400000,24.500000,24.500000,24.500000,0
1993-04-28,24.500000,24.600000,23.700001,24.299999,24.299999,0
1993-04-29,24.299999,24.799999,23.700001,23.700001,23.700001,0
1993-04-30,23.700001,24.200001,23.700001,24.100000,24.100000,0
1993-05-03,24.100000,24.400000,23.799999,24.100000,24.100000,0
...,...,...,...,...,...,...
2023-09-25,116009.000000,116031.000000,115573.000000,115925.000000,115925.000000,9580000
2023-09-26,115922.000000,115922.000000,114162.000000,114193.000000,114193.000000,11241800
2023-09-27,114194.000000,115340.000000,113366.000000,114327.000000,114327.000000,13359300


In [113]:
ibovespa = ibovespa.drop(['Volume','Adj Close'],axis=1)
ibovespa

,Open,High,Low,Close
Date,,,,
1993-04-27,24.799999,25.400000,24.500000,24.500000
1993-04-28,24.500000,24.600000,23.700001,24.299999
1993-04-29,24.299999,24.799999,23.700001,23.700001
1993-04-30,23.700001,24.200001,23.700001,24.100000
1993-05-03,24.100000,24.400000,23.799999,24.100000
...,...,...,...,...
2023-09-25,116009.000000,116031.000000,115573.000000,115925.000000
2023-09-26,115922.000000,115922.000000,114162.000000,114193.000000
2023-09-27,114194.000000,115340.000000,113366.000000,114327.000000


In [114]:
ibovespa['Tomorrow Close'] = ibovespa['Close'].shift(-1)
ibovespa

,Open,High,Low,Close,Tomorrow Close
Date,,,,,
1993-04-27,24.799999,25.400000,24.500000,24.500000,24.299999
1993-04-28,24.500000,24.600000,23.700001,24.299999,23.700001
1993-04-29,24.299999,24.799999,23.700001,23.700001,24.100000
1993-04-30,23.700001,24.200001,23.700001,24.100000,24.100000
1993-05-03,24.100000,24.400000,23.799999,24.100000,24.900000
...,...,...,...,...,...
2023-09-25,116009.000000,116031.000000,115573.000000,115925.000000,114193.000000
2023-09-26,115922.000000,115922.000000,114162.000000,114193.000000,114327.000000
2023-09-27,114194.000000,115340.000000,113366.000000,114327.000000,115731.000000


In [115]:
ibovespa['Increased'] = (ibovespa['Tomorrow Close'] > ibovespa['Close']).astype(int)
ibovespa

,Open,High,Low,Close,Tomorrow Close,Increased
Date,,,,,,
1993-04-27,24.799999,25.400000,24.500000,24.500000,24.299999,0
1993-04-28,24.500000,24.600000,23.700001,24.299999,23.700001,0
1993-04-29,24.299999,24.799999,23.700001,23.700001,24.100000,1
1993-04-30,23.700001,24.200001,23.700001,24.100000,24.100000,0
1993-05-03,24.100000,24.400000,23.799999,24.100000,24.900000,1
...,...,...,...,...,...,...
2023-09-25,116009.000000,116031.000000,115573.000000,115925.000000,114193.000000,0
2023-09-26,115922.000000,115922.000000,114162.000000,114193.000000,114327.000000,1
2023-09-27,114194.000000,115340.000000,113366.000000,114327.000000,115731.000000,1


# Technical Analysis Indicators

In [116]:
time_periods = [3,7,10]

ibovespa_Cont = pd.DataFrame()

for time_period in time_periods:

  weights = list(range(1, time_period+1))
  # Defina uma função personalizada para calcular a média ponderada
  def weighted_average(values):
      return (values * weights).sum() / sum(weights)
  rolling_weighted_averages = ibovespa.rolling(time_period).apply(weighted_average, raw=True)

  ######################################################################################

  # Calcule as mudanças de preço diárias
  ibovespa['Price Change'] = ibovespa['Close'].diff()

  # Separe as mudanças de preço positivas e negativas
  ibovespa['Positive Change'] = np.where(ibovespa['Price Change'] > 0, ibovespa['Price Change'], 0)
  ibovespa['Negative Change'] = np.where(ibovespa['Price Change'] < 0, -ibovespa['Price Change'], 0)

  # Calcule o RS (Relative Strength)
  rs = ibovespa['Positive Change'].rolling(time_period).mean() / ibovespa['Negative Change'].rolling(time_period).mean()

  # Calcule o RSI usando a fórmula
  rsi = 100 - (100 / (1 + rs))

  ######################################################################################

  # Especifique os períodos para o cálculo do MACD e da linha de sinal
  periodo_curto = time_period + 9
  periodo_longo = time_period + 23
  periodo_sinal = time_period + 6

  # Calcule as EMA12 e EMA26
  ema12 = ibovespa['Close'].ewm(span=periodo_curto, adjust=False).mean()
  ema26 = ibovespa['Close'].ewm(span=periodo_longo, adjust=False).mean()

  # Calcule o MACD
  macd = ema12 - ema26

  ######################################################################################

  signal_line = macd.ewm(span=periodo_sinal, adjust=False).mean()

  ######################################################################################

  rolling_max = ibovespa['High'].rolling(time_period).max()
  rolling_min = ibovespa['Low'].rolling(time_period).min()

  ######################################################################################

  # Calcule o Preço Típico
  ibovespa['Typical Price'] = (ibovespa['High'] + ibovespa['Low'] + ibovespa['Close']) / 3

  # Calcule a Média Móvel do Preço Típico
  ibovespa['Typical Price Moving Average'] = ibovespa['Typical Price'].rolling(time_period).mean()

  # Calcule o Desvio Padrão do Preço Típico
  ibovespa['Typical Price Std'] = ibovespa['Typical Price'].rolling(time_period).std()

  ######################################################################################

  ibovespa_Cont[f'simple_{time_period}days_moving_average'] = ibovespa['Close'].rolling(time_period).mean()
  ibovespa_Cont[f'weighted_{time_period}days_moving_average'] = rolling_weighted_averages['Close']
  ibovespa_Cont[f'{time_period}days_momentum'] = ibovespa['Close'].rolling(time_period).apply(lambda x: x[-1] - x[0])
  ibovespa_Cont[f'{time_period}days_stochastic_K'] = 100*(ibovespa['Close'] - rolling_min) / (rolling_max - rolling_min)
  ibovespa_Cont[f'{time_period}days_stochastic_D'] = ibovespa_Cont[f'{time_period}days_stochastic_K'].rolling(time_period).mean()
  ibovespa_Cont[f'{time_period}days_RSI'] = rsi
  ibovespa_Cont[f'{time_period}days_MACD'] = macd
  ibovespa_Cont[f'{time_period}days_Signal_Line'] = signal_line
  ibovespa_Cont[f'{time_period}days_LW_R'] = 100*(rolling_max - ibovespa['Close']) / (rolling_max - rolling_min)
  ibovespa_Cont[f'{time_period}days_CCI'] = (ibovespa['Typical Price'] - ibovespa['Typical Price Moving Average']) / (0.015 * ibovespa['Typical Price Std'])

  #####################################################################################

ibovespa_Cont = ibovespa_Cont.dropna()

In [117]:
ibovespa_Cont = ibovespa_Cont['2020-01-01':]

In [118]:
df_x = ibovespa_Cont
df_x

,simple_3days_moving_average,weighted_3days_moving_average,3days_momentum,3days_stochastic_K,3days_stochastic_D,3days_RSI,3days_MACD,3days_Signal_Line,3days_LW_R,3days_CCI,...,simple_10days_moving_average,weighted_10days_moving_average,10days_momentum,10days_stochastic_K,10days_stochastic_D,10days_RSI,10days_MACD,10days_Signal_Line,10days_LW_R,10days_CCI
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,117023.666667,117363.500000,2039.0,100.000000,56.321045,67.801455,2373.626214,1971.690129,0.000000,71.408683,...,115321.6,116245.072727,6677.0,100.000000,90.939106,80.516050,1892.869919,1389.749660,0.000000,90.860845
2020-01-03,117414.666667,117705.166667,1743.0,65.478842,59.713605,64.499382,2409.093513,2059.170806,34.521158,50.934747,...,115902.7,116678.781818,5091.0,84.263959,91.963865,78.936361,1945.492848,1455.131212,15.736041,86.941298
2020-01-06,117719.333333,117436.833333,-1695.0,39.102768,68.193870,60.618030,2343.296172,2115.995879,60.897232,-72.258477,...,116328.9,116856.109091,2563.0,70.517560,90.387521,70.995074,1944.488194,1512.702621,29.482440,41.056211
2020-01-07,117082.333333,116908.166667,-1045.0,24.655111,43.078907,0.000000,2247.810550,2142.358813,75.344889,-55.033878,...,116563.6,116916.672727,1531.0,58.070866,86.250023,63.539864,1923.304910,1561.008773,41.929134,10.580794
2020-01-08,116595.666667,116490.500000,-631.0,27.507448,30.421776,0.000000,2114.278389,2136.742728,72.492552,-54.615252,...,116675.2,116859.109091,1126.0,40.342241,80.286247,56.750544,1876.959629,1598.179462,59.657759,-11.374234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-25,116026.333333,115989.666667,-220.0,11.274824,6.344364,0.000000,-185.594142,-42.275333,88.725176,-61.391004,...,117720.2,117261.818182,-2043.0,8.367007,58.377888,43.758144,-108.194363,-42.012157,91.632993,-116.274038
2023-09-26,115375.666667,115073.000000,-1816.0,1.104775,5.491418,0.000000,-409.654833,-115.751233,98.895225,-73.952006,...,117342.7,116620.509091,-3983.0,0.551798,50.614477,27.316428,-229.104008,-64.022963,99.448202,-123.273190
2023-09-27,114815.000000,114548.666667,-1598.0,36.060038,16.146546,6.871795,-569.843068,-206.569600,63.939962,-54.944568,...,116957.8,116072.200000,-5065.0,14.982850,44.662438,26.664242,-324.656272,-94.685705,85.017150,-104.957494


In [119]:
df_y = ibovespa['Increased']['2020-01-01':]
df_y

Date
2020-01-02    0
2020-01-03    0
2020-01-06    0
2020-01-07    0
2020-01-08    0
             ..
2023-09-25    0
2023-09-26    1
2023-09-27    1
2023-09-28    1
2023-09-29    0
Name: Increased, Length: 933, dtype: int64

In [120]:
predictors = df_x.columns

train_size = int(round(len(df_y)*0.7,0))
test_size = len(df_y) - train_size

# Splits the X dataset into train and test
x_train = np.array(df_x[0:train_size])
x_test = np.array(df_x[train_size:])

# Splits the Y dataset into train and test
y_train = np.array(df_y[:train_size])
y_test = np.array(df_y[train_size:])

# Data Resizing

In [121]:
n_agg = 10
x_agg = []

for i in range(len(df_x)-n_agg+1):
  x_agg.append(df_x[i:i+n_agg])

x_agg = np.array(x_agg)

In [122]:
train_size = int(round(x_agg.shape[0]*0.7,0))
test_size = 1 - train_size

In [123]:
x_train_agg = x_agg[:train_size]
x_test_agg = x_agg[train_size:]

In [124]:
y_agg = df_y[n_agg-1:]

In [125]:
from scipy.special import y1
y_train_agg = y_agg[:train_size]
y_test_agg = y_agg[train_size:]

In [126]:
x_train_agg.shape, y_train_agg.shape

((647, 10, 30), (647,))

In [127]:
x_test_agg.shape, y_test_agg.shape

((277, 10, 30), (277,))

#LSTM

In [128]:
from tensorflow.keras.layers import LSTM

n = 1
layer_1 = [8, 16, 24, 32, 64, 128]
layer_2 = [4, 8, 12, 16, 32, 64]

for i,j in zip(layer_1, layer_2):

    print(f'LSTM {n}')
    print('Layer 1 = ', i)
    print('Layer 2 = ', j)

    # Creates the MLP with 2 hidden layers
    LSTM_model = Sequential([
        LSTM(i, activation='relu', input_shape=(x_train_agg.shape[1], x_train_agg.shape[2]), return_sequences=True),
        LSTM(j, activation='relu', return_sequences=False),
        Dropout(0.2),
        Dense(1)])

    # Compiles the model
    LSTM_model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['accuracy'])

    LSTM_model.fit(x_train_agg, y_train_agg.values, validation_split=0.1, epochs=10, batch_size=16, verbose=0)

    start_time = time.time()
    y_pred = LSTM_model.predict(x_test_agg)
    end_time = time.time()

    threshold = 0.5
    y_pred = (y_pred > threshold).astype(int)

    accuracy = accuracy_score(y_test_agg, y_pred)
    print('Accuracy: '+str(round(accuracy,4)))

    # Obtendo valores da matriz de confusão
    tn, fp, fn, tp = confusion_matrix(y_test_agg, y_pred).ravel()
    # Calculando a True Positive Rate (Taxa de Verdadeiros Positivos)
    tpr = round(tp / (tp + fp),4)
    print("True Positive Rate:", round(tpr,4))

    execution_time = end_time - start_time
    print("Tempo de execução:", round(execution_time, 4), "segundos")
    print('')

    n+=1

LSTM 1
Layer 1 =  8
Layer 2 =  4
9/9 [==============================] - 0s 2ms/step
Accuracy: 0.5199
True Positive Rate: nan
Tempo de execução: 0.2716 segundos

LSTM 2
Layer 1 =  16
Layer 2 =  8


<ipython-input-128-41af981fd41d>:40: RuntimeWarning: invalid value encountered in long_scalars
  tpr = round(tp / (tp + fp),4)


9/9 [==============================] - 0s 2ms/step
Accuracy: 0.5199
True Positive Rate: nan
Tempo de execução: 0.2646 segundos

LSTM 3
Layer 1 =  24
Layer 2 =  12


<ipython-input-128-41af981fd41d>:40: RuntimeWarning: invalid value encountered in long_scalars
  tpr = round(tp / (tp + fp),4)


9/9 [==============================] - 0s 3ms/step
Accuracy: 0.5271
True Positive Rate: 1.0
Tempo de execução: 0.3858 segundos

LSTM 4
Layer 1 =  32
Layer 2 =  16
9/9 [==============================] - 0s 2ms/step
Accuracy: 0.5776
True Positive Rate: 0.58
Tempo de execução: 0.2445 segundos

LSTM 5
Layer 1 =  64
Layer 2 =  32
9/9 [==============================] - 0s 3ms/step
Accuracy: 0.5235
True Positive Rate: 0.5075
Tempo de execução: 0.2989 segundos

LSTM 6
Layer 1 =  128
Layer 2 =  64
9/9 [==============================] - 0s 4ms/step
Accuracy: 0.4729
True Positive Rate: 0.4711
Tempo de execução: 0.2928 segundos



#GRU

In [129]:
# Implements the random forest model one more time, but with different parameters

from tensorflow.keras.layers import GRU

n = 1
layer_1 = [8, 16, 24, 32, 64, 128]
layer_2 = [4, 8, 12, 16, 32, 64]

for i,j in zip(layer_1, layer_2):

    print(f'GRU {n}')
    print('Layer 1 = ', i)
    print('Layer 2 = ', j)

    # Creates the MLP with 2 hidden layers
    GRU_model = Sequential([
        GRU(i, activation='relu', input_shape=(x_train_agg.shape[1], x_train_agg.shape[2]), return_sequences=True),
        GRU(j, activation='relu', return_sequences=False),
        Dropout(0.2),
        Dense(1)])

    # Compiles the model
    GRU_model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['accuracy'])

    GRU_model.fit(x_train_agg, y_train_agg.values, validation_split=0.1, epochs=10, batch_size=16, verbose=0)

    start_time = time.time()
    y_pred = GRU_model.predict(x_test_agg)
    end_time = time.time()

    threshold = 0.5
    y_pred = (y_pred > threshold).astype(int)

    accuracy = accuracy_score(y_test_agg, y_pred)
    print('Accuracy: '+str(round(accuracy,4)))

    # Obtendo valores da matriz de confusão
    tn, fp, fn, tp = confusion_matrix(y_test_agg, y_pred).ravel()
    # Calculando a True Positive Rate (Taxa de Verdadeiros Positivos)
    tpr = round(tp / (tp + fp),4)
    print("True Positive Rate:", tpr)

    execution_time = end_time - start_time
    print("Tempo de execução:", round(execution_time, 4), "segundos")
    print('')

    n+=1

GRU 1
Layer 1 =  8
Layer 2 =  4
9/9 [==============================] - 0s 2ms/step
Accuracy: 0.5199
True Positive Rate: nan
Tempo de execução: 0.3032 segundos

GRU 2
Layer 1 =  16
Layer 2 =  8


<ipython-input-129-c138ff985be1>:42: RuntimeWarning: invalid value encountered in long_scalars
  tpr = round(tp / (tp + fp),4)


9/9 [==============================] - 0s 3ms/step
Accuracy: 0.5199
True Positive Rate: nan
Tempo de execução: 0.3638 segundos

GRU 3
Layer 1 =  24
Layer 2 =  12


<ipython-input-129-c138ff985be1>:42: RuntimeWarning: invalid value encountered in long_scalars
  tpr = round(tp / (tp + fp),4)


9/9 [==============================] - 0s 2ms/step
Accuracy: 0.5162
True Positive Rate: 0.4962
Tempo de execução: 0.306 segundos

GRU 4
Layer 1 =  32
Layer 2 =  16
9/9 [==============================] - 0s 2ms/step
Accuracy: 0.5199
True Positive Rate: nan
Tempo de execução: 0.2971 segundos

GRU 5
Layer 1 =  64
Layer 2 =  32


<ipython-input-129-c138ff985be1>:42: RuntimeWarning: invalid value encountered in long_scalars
  tpr = round(tp / (tp + fp),4)


9/9 [==============================] - 0s 3ms/step
Accuracy: 0.4801
True Positive Rate: 0.4801
Tempo de execução: 0.2981 segundos

GRU 6
Layer 1 =  128
Layer 2 =  64
9/9 [==============================] - 0s 4ms/step
Accuracy: 0.4765
True Positive Rate: 0.4756
Tempo de execução: 0.3167 segundos



# LSTM-GRU

In [135]:
from tensorflow.keras.layers import GRU, LSTM

n = 1
layer_1 = [8, 16, 24, 32, 64, 128]
layer_2 = [4, 8, 12, 16, 32, 64]

for i,j in zip(layer_1, layer_2):

    print(f'LSTM-GRU {n}')
    print('Layer 1 = ', i)
    print('Layer 2 = ', j)

    # Crie o modelo
    LSTM_GRU_model = Sequential([
        LSTM(i, activation='relu', input_shape=(x_train_agg.shape[1], x_train_agg.shape[2]), return_sequences=True),
        GRU(j, activation='relu'),
        Dropout(0.2),
        Dense(1)])

    # Compile o modelo
    LSTM_GRU_model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['accuracy'])  # Substitua a função de perda conforme necessário

    LSTM_GRU_model.fit(x_train_agg, y_train_agg.values, validation_split=0.1, epochs=10, batch_size=16, verbose=0)

    y_pred = LSTM_GRU_model.predict(x_test_agg)
    threshold = 0.5
    y_pred = (y_pred > threshold).astype(int)

    accuracy = accuracy_score(y_test_agg, y_pred)
    print('Accuracy: '+str(round(accuracy,4)))

    # Obtendo valores da matriz de confusão
    tn, fp, fn, tp = confusion_matrix(y_test_agg, y_pred).ravel()
    # Calculando a True Positive Rate (Taxa de Verdadeiros Positivos)
    tpr = round(tp / (tp + fp),4)
    print("True Positive Rate:", tpr)
    print('')

    n+=1

LSTM-GRU 1
Layer 1 =  8
Layer 2 =  4
9/9 [==============================] - 0s 2ms/step
Accuracy: 0.5126
True Positive Rate: 0.375

LSTM-GRU 2
Layer 1 =  16
Layer 2 =  8
9/9 [==============================] - 0s 4ms/step
Accuracy: 0.5271
True Positive Rate: 0.625

LSTM-GRU 3
Layer 1 =  24
Layer 2 =  12
9/9 [==============================] - 0s 2ms/step
Accuracy: 0.4801
True Positive Rate: 0.4801

LSTM-GRU 4
Layer 1 =  32
Layer 2 =  16
9/9 [==============================] - 0s 2ms/step
Accuracy: 0.4874
True Positive Rate: 0.4759

LSTM-GRU 5
Layer 1 =  64
Layer 2 =  32
9/9 [==============================] - 0s 3ms/step
Accuracy: 0.4621
True Positive Rate: 0.4588

LSTM-GRU 6
Layer 1 =  128
Layer 2 =  64
9/9 [==============================] - 0s 5ms/step
Accuracy: 0.5379
True Positive Rate: 0.5214



# GRU-LSTM

In [139]:
from tensorflow.keras.layers import GRU, LSTM

n = 1
layer_1 = [8, 16, 24, 32, 64, 128]
layer_2 = [4, 8, 12, 16, 32, 64]

for i,j in zip(layer_1, layer_2):

    print(f'GRU-LSTM {n}')
    print('Layer 1 = ', i)
    print('Layer 2 = ', j)

    # Crie o modelo
    LSTM_GRU_model = Sequential([
        GRU(i, activation='relu', input_shape=(x_train_agg.shape[1], x_train_agg.shape[2]), return_sequences=True),
        LSTM(j, activation='relu'),
        Dropout(0.2),
        Dense(1)])

    # Compile o modelo
    LSTM_GRU_model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['accuracy'])  # Substitua a função de perda conforme necessário

    LSTM_GRU_model.fit(x_train_agg, y_train_agg.values, validation_split=0.1, epochs=10, batch_size=16, verbose=0)

    y_pred = LSTM_GRU_model.predict(x_test_agg)
    threshold = 0.5
    y_pred = (y_pred > threshold).astype(int)

    accuracy = accuracy_score(y_test_agg, y_pred)
    print('Accuracy: '+str(round(accuracy,4)))

    # Obtendo valores da matriz de confusão
    tn, fp, fn, tp = confusion_matrix(y_test_agg, y_pred).ravel()
    # Calculando a True Positive Rate (Taxa de Verdadeiros Positivos)
    tpr = round(tp / (tp + fp),4)
    print("True Positive Rate:", tpr)
    print('')

    n+=1

GRU-LSTM 1
Layer 1 =  8
Layer 2 =  4
9/9 [==============================] - 0s 2ms/step
Accuracy: 0.5199
True Positive Rate: nan

GRU-LSTM 2
Layer 1 =  16
Layer 2 =  8


<ipython-input-139-632e6328da33>:37: RuntimeWarning: invalid value encountered in long_scalars
  tpr = round(tp / (tp + fp),4)


9/9 [==============================] - 0s 2ms/step
Accuracy: 0.5199
True Positive Rate: nan

GRU-LSTM 3
Layer 1 =  24
Layer 2 =  12


<ipython-input-139-632e6328da33>:37: RuntimeWarning: invalid value encountered in long_scalars
  tpr = round(tp / (tp + fp),4)


9/9 [==============================] - 0s 2ms/step
Accuracy: 0.5199
True Positive Rate: nan

GRU-LSTM 4
Layer 1 =  32
Layer 2 =  16


<ipython-input-139-632e6328da33>:37: RuntimeWarning: invalid value encountered in long_scalars
  tpr = round(tp / (tp + fp),4)


9/9 [==============================] - 0s 2ms/step
Accuracy: 0.5271
True Positive Rate: 0.5333

GRU-LSTM 5
Layer 1 =  64
Layer 2 =  32
9/9 [==============================] - 0s 3ms/step
Accuracy: 0.4838
True Positive Rate: 0.48

GRU-LSTM 6
Layer 1 =  128
Layer 2 =  64
9/9 [==============================] - 0s 3ms/step
Accuracy: 0.5018
True Positive Rate: 0.4884



# Benchmark

In [137]:
true_ratio = round(len(y_test[y_test == 1])/len(y_test),4)
true_ratio

0.4786